In [1]:
import pandas as pd
import numpy as np
import bz2

In [2]:
def get_labels_texts(file):
    labels = []
    texts  = []
    for line in bz2.BZ2File(file):
        x = line.decode('utf-8')
        labels.append(int(x[9])-1)
        texts.append(x[10:].strip())
        
    return np.array(labels), texts

train_labels, train_texts = get_labels_texts("../Data/amazon/train.ft.txt.bz2")
test_labels, test_texts = get_labels_texts("../Data/amazon/test.ft.txt.bz2")

In [3]:
train_labels[0]

1

In [4]:
train_texts[0]

'Stuning even for the non-gamer: This sound track was beautiful! It paints the senery in your mind so well I would recomend it even to people who hate vid. game music! I have played the game Chrono Cross but out of all of the games I have ever played it has the best music! It backs away from crude keyboarding and takes a fresher step with grate guitars and soulful orchestras. It would impress anyone who cares to listen! ^_^'

In [5]:
small_train_labels = train_labels[0:500]
small_train_texts  = train_texts[0:500]

## Text PreProcessing

In [6]:
import re

NON_ALPHANUMERIC = re.compile(r'[\W]')
NON_ASCII        = re.compile(r'[^a-z0-1\S]')

def normalize_text(texts):
    normalized_list = []
    for text in texts:
        lower = text.lower()
        no_punctuation = NON_ALPHANUMERIC.sub(r' ',lower)
        no_non_ascii   = NON_ASCII.sub(r' ',no_punctuation)
        
        # split and join to remove extra spaces between words
        clean_text = no_non_ascii.split(" ")
        clean_text = " ".join([word for word in clean_text if word])
        
        normalized_list.append(clean_text)
        
    return normalized_list

In [7]:
small_train_texts = normalize_text(small_train_texts)

In [8]:
small_train_texts[0]

'stuning even for the non gamer this sound track was beautiful it paints the senery in your mind so well i would recomend it even to people who hate vid game music i have played the game chrono cross but out of all of the games i have ever played it has the best music it backs away from crude keyboarding and takes a fresher step with grate guitars and soulful orchestras it would impress anyone who cares to listen _'

## Count Vectorizer

In [9]:
from sklearn.feature_extraction.text import CountVectorizer 

cv = CountVectorizer(binary=True)

In [10]:
cv.fit(small_train_texts)

CountVectorizer(binary=True)

In [11]:
X = cv.transform(small_train_texts)

In [12]:
X_test  =  cv.transform(test_texts)

In [13]:
X_test

<400000x5853 sparse matrix of type '<class 'numpy.int64'>'
	with 18679328 stored elements in Compressed Sparse Row format>

In [18]:
from sklearn.linear_model import LogisticRegression 
from sklearn.metrics import accuracy_score 
from sklearn.model_selection import train_test_split 

X_train , X_val, y_train, y_val = train_test_split(X, small_train_labels, train_size = 0.75)

for c in [0.01, 0.025, 0.05, 0.75, 1]:
    lr = LogisticRegression(C=c)
    lr.fit(X_train, y_train)
    print(f'Accuracy for C:{c}, {accuracy_score(y_val, lr.predict(X_val))}')

Accuracy for C:0.01, 0.76
Accuracy for C:0.025, 0.752
Accuracy for C:0.05, 0.752
Accuracy for C:0.75, 0.768
Accuracy for C:1, 0.768


In [23]:
lr.predict(X_test[9])

array([1])

In [24]:
test_labels[9]

0

# TEST CASE

In [34]:
test  =  cv.transform(['this is a not a good product!, but it crashed the next day making my heart go'])

In [41]:
lr.predict(X_test[:50])

array([1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0,
       1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1,
       0, 0, 0, 0, 0, 0])

In [44]:
test_labels[0:50]

array([1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0,
       1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1,
       1, 1, 0, 0, 0, 1])

In [47]:
accuracy_score(test_labels[150:250], lr.predict(X_test[150:250]))

0.73